# 第10章: 事前学習済み言語モデル（GPT型）

本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

## 90. 次単語予測

“The movie was full of"に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [1]:
!uv pip install openai
!uv pip install python-dotenv
!uv pip install tiktoken

Using Python 3.11.10 environment at: /Users/ryuichi/.venv
Resolved 16 packages in 1.87s                                        
Prepared 7 packages in 2.17s                                             
Installed 7 packages in 39ms                                
 + annotated-types==0.7.0
 + distro==1.9.0
 + jiter==0.9.0
 + openai==1.77.0
 + pydantic==2.11.4
 + pydantic-core==2.33.2
 + typing-inspection==0.4.0
Using Python 3.11.10 environment at: /Users/ryuichi/.venv
Resolved 1 package in 329ms                                          
Prepared 1 package in 77ms                                               
Installed 1 package in 40ms                                 
 + python-dotenv==1.1.0
Using Python 3.11.10 environment at: /Users/ryuichi/.venv
Resolved 7 packages in 677ms                                         
Prepared 1 package in 441ms                                              
Installed 1 package in 29ms                                 
 + tiktoken==0.9.0


In [2]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# .envファイルをロードして環境変数を読み込む
load_dotenv()

# 環境変数から値を取得
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# 必須の環境変数が欠けている場合エラーをスロー
if not azure_endpoint or not api_key or not api_version:
    raise ValueError("必須の環境変数の値が取得できていません。環境変数を確認してください。")

# Azure OpenAI Clientの初期化
client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version=api_version
)

In [6]:
import textwrap
import tiktoken # tiktoken ライブラリをインポート

def fetch_top_tokens_and_probabilities(payload, system_message="あなたは優秀な映画の専門家です。"):
    """
    Azure OpenAIのモデルに対してプロンプトに続くトークンとその確率を取得します。

    Parameters:
        - payload (dict): モデル名、プロンプト、温度、top_kを含む辞書。
        - system_message (str): システムメッセージ（デフォルト値あり）。

    Returns:
        tuple: (APIが実際に生成した最初のトークン文字列, 上位トークンとその確率のリスト)
    """
    try:
        # API呼び出し: 次のトークン予測
        # logprobs=True に加えて、top_logprobs パラメータを追加
        response = client.chat.completions.create(
            model=payload["model"],
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": payload["prompt"]}
            ],
            temperature=payload["temperature"],
            max_tokens=1, # 続く「一つの」トークンを見るため max_tokens=1
            logprobs=True,
            top_logprobs=payload["top_k"] # 返却する上位トークン数を指定
        )

        # print(f"APIレスポンス: {response}") # デバッグが必要な場合のみ表示推奨

        response_dict = response.model_dump()

        # max_tokens=1 なので、生成された最初のトークンを取得
        generated_token_text = response_dict["choices"][0]["message"]["content"]

        # 上位トークンとその確率を取得
        # logprobs -> content -> [0] (最初の出力トークン) -> top_logprobs にアクセス
        # top_logprobs はリスト内の辞書です: [{'token': ' abc', 'logprob': -0.1, 'bytes': [...]}, ...]
        # content リストが空でないかチェック（max_tokens=1なので通常は1つの要素があるはず）
        if not response_dict["choices"][0].get("logprobs") or not response_dict["choices"][0]["logprobs"].get("content"):
             print("警告: APIレスポンスにlogprobs情報が含まれていませんでした。")
             return generated_token_text, []

        # 最初の生成トークン位置での上位候補トークン情報を取得
        top_logprobs_list_of_dicts = response_dict["choices"][0]["logprobs"]["content"][0]["top_logprobs"]

        if not top_logprobs_list_of_dicts:
             print("警告: APIから上位トークン情報が返されませんでした。top_logprobs=0 の可能性があります。")
             return generated_token_text, []


        top_tokens_and_probabilities = [
            (token_info["token"], round(pow(10, token_info["logprob"]), 5))
            for token_info in top_logprobs_list_of_dicts
        ]

        # APIが top_logprobs で指定した数のトークンを確率付きで返しているので、ソートして返します
        return generated_token_text, sorted(top_tokens_and_probabilities, key=lambda x: x[1], reverse=True)

    except Exception as e:
        print(f"API呼び出し中にエラーが発生しました: {e}")
        # エラー発生時は生成されたトークンも確率リストも取得できない
        return None, None

payload = {
    "model": "gpt-4o",  # あなたのAzure OpenAIでデプロイしたモデルのデプロイメント名を指定
    # "prompt": "The movie was full of",
    "prompt": "The movie was full of",
    "temperature": 0.7, # Temperature affects sampling of the *generated* token, but logprobs shows potential options regardless.
    "top_k": 10 # APIに返すように要求する上位トークンの数
}

print(f"「'{payload['prompt']}'」に続くトークン上位{payload['top_k']}個と、その確率（尤度）を求めよ。")
print(f"ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。")

# --- プロンプトのトークン化を表示 ---
print("\n--- プロンプトのトークン化結果 ---")
try:
    # モデル名に対応するエンコーディングを取得
    encoding = tiktoken.encoding_for_model(payload["model"])

    # プロンプトをトークンIDのリストにエンコード
    input_token_ids = encoding.encode(payload["prompt"])

    # 各トークンIDを元のテキストに戻す（デコード）
    # 各IDごとにデコードすると、元の区切り（スペースなど）を維持しやすい
    input_tokens_text = [encoding.decode([token_id]) for token_id in input_token_ids]

    print(f"元のプロンプト: '{payload['prompt']}'")
    print(f"トークンID列: {input_token_ids}")
    print(f"トークンテキスト列: {input_tokens_text}")
    print(f"トークン数: {len(input_token_ids)}")

except Exception as e:
    print(f"プロンプトのトークン化中にエラーが発生しました: {e}")
    print("tiktoken がインストールされているか確認してください (`pip install tiktoken`)。")


# --- 続くトークンの確率を取得 ---
print("\n--- 続くトークン候補と確率 ---")
# APIを呼び出して続くトークンの確率を取得
generated_token, top_tokens_and_probabilities = fetch_top_tokens_and_probabilities(payload)

# 結果を表示
if generated_token is not None and top_tokens_and_probabilities is not None:
    # API呼び出しによって実際に生成された最初のトークン
    # これは logprobs リストに含まれる可能性が高いですが、サンプリングによる結果です。
    print(f"API呼び出しで実際に生成された最初のトークン (max_tokens=1 の結果): '{generated_token}'")

    print(f"\n'{payload['prompt']}' に続く位置でのトークン候補上位{payload['top_k']}件とその確率:")
    if top_tokens_and_probabilities:
        # 確率の高い順に表示
        for token, probability in top_tokens_and_probabilities:
            print(f"  '{token}': {probability}") # トークンを引用符で囲むと見やすい
    else:
        print("上位トークン候補情報は取得できませんでした。")

「'The movie was full of'」に続くトークン上位10個と、その確率（尤度）を求めよ。
ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

--- プロンプトのトークン化結果 ---
元のプロンプト: 'The movie was full of'
トークンID列: [976, 8249, 673, 3149, 328]
トークンテキスト列: ['The', ' movie', ' was', ' full', ' of']
トークン数: 5

--- 続くトークン候補と確率 ---
API呼び出しで実際に生成された最初のトークン (max_tokens=1 の結果): 'The'

'The movie was full of' に続く位置でのトークン候補上位10件とその確率:
  'The': 0.17045
  'It': 0.00539
  'Could': 0.00303
  'intr': 0.00096
  'st': 0.00054
  'unexpected': 0.0003
  'em': 0.0003
  'Can': 0.00017
  'v': 0.0001
  'spect': 0.0001


## 91. 続きのテキストの予測

“The movie was full of"に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。

In [7]:
import textwrap
import tiktoken
import os # 環境変数からAPIキーなどを読み込む場合に使用

# --- テキスト補完生成関数 ---
def generate_text_completions(client, model, prompt, max_tokens, temperature, top_p, n):
    """
    指定されたパラメータでAzure OpenAIモデルから複数のテキスト補完を生成します。

    Parameters:
        - client: AzureOpenAI クライアントインスタンス
        - model (str): モデル名
        - prompt (str): 入力プロンプト
        - max_tokens (int): 生成する最大トークン数 (1以上)
        - temperature (float): サンプリング温度 (0.0-2.0)。0で決定論的。
        - top_p (float): top_p サンプリング確率 (0.0-1.0)。1.0でtemperatureサンプリングに近い。
        - n (int): 生成する補完の数 (1以上)

    Returns:
        list: 生成されたテキスト文字列のリスト。エラー発生時はエラーメッセージを含むリスト。
    """
    if client is None:
        return ["Error: Azure OpenAI client is not initialized."] * n

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            n=n, # 複数の補完を生成
            # logprobs, top_logprobs はここでは使用しない
        )

        # 生成されたテキストをリストとして抽出
        completions = [choice.message.content for choice in response.choices]

        return completions

    except Exception as e:
        print(f"API呼び出し中にエラーが発生しました (温度={temperature}, top_p={top_p}): {e}")
        return [f"Error generating completion: {e}"] * n # エラー時はエラーメッセージを含むリストを返す

# --- 主処理 ---

model_name = "gpt-4o"
prompt_text = "The movie was full of"
generate_max_tokens = 100 # 生成するテキストの最大長（トークン単位）。必要に応じて調整。

# 試行する温度とtop_pの値のリスト
# 温度 (Temperature): 高いほどランダム性が増す (多様だが脱線も)。低いほど決定論的 (毎回同じ傾向)。
temperatures_to_test = [0.0, 0.7, 1.0, 1.5]
# top_p (Nucleus Sampling): 累積確率が top_p になるまで確率の高いトークン候補を選び、その中からサンプリング。
# 低いほど候補が絞られ予測可能に。高いほど候補が増え多様に (1.0 は temperature サンプリングに近い)。
top_p_to_test = [0.1, 0.5, 1.0]

# 各設定で生成するテキストの数
num_completions_per_setting = 3 # 各温度・top_pの組み合わせで3つの補完を生成

print(f"=== 課題 ===")
print(f"「'{prompt_text}'」に続くテキストを複数予測し、デコーディング方法（温度、top_p）による変化を観察する。")
print(f"モデル: {model_name}")
print(f"生成最大トークン数: {generate_max_tokens}")
print("============")

print("\n=== テキスト補完の生成と観察 ===")

# 温度とtop_pの組み合わせごとに補完を生成
for temp in temperatures_to_test:
    for p in top_p_to_test:
        # temperature=0.0 の場合、top_p は通常無視され決定論的なサンプリングになります。
        # 重複を避けるため、temperature=0.0 の場合は top_p=1.0 の設定のみ実行します。
        if temp == 0.0 and p != 1.0:
            continue # 他の top_p はスキップ

        # 設定の表示
        if temp == 0.0:
             print(f"\n--- 設定: 温度={temp} (決定論的サンプリング) ---")
        else:
             print(f"\n--- 設定: 温度={temp}, top_p={p} ---")

        # テキスト補完を生成
        completions = generate_text_completions(
            client, # クライアントオブジェクトを渡す
            model_name,
            prompt_text,
            generate_max_tokens,
            temp,
            p,
            num_completions_per_setting
        )

        # 生成された補完を表示
        for i, text in enumerate(completions):
            print(f"  補完 {i+1}:")
            # テキストを整形して表示
            print(textwrap.fill(text, width=80, initial_indent='    ', subsequent_indent='    '))

=== 課題 ===
「'The movie was full of'」に続くテキストを複数予測し、デコーディング方法（温度、top_p）による変化を観察する。
モデル: gpt-4o
生成最大トークン数: 100

=== テキスト補完の生成と観察 ===

--- 設定: 温度=0.0 (決定論的サンプリング) ---
  補完 1:
    The movie was full of **unexpected twists**, **emotional depth**, and
    **stunning visuals** that kept the audience engaged from start to finish.
    Whether it was the gripping storyline, the well-developed characters, or the
    breathtaking cinematography, it delivered an unforgettable experience. What
    kind of movie are you referring to?
  補完 2:
    The movie was full of **unexpected twists**, **emotional depth**, and
    **stunning visuals** that kept the audience engaged from start to finish.
    Whether it was the gripping storyline, the well-developed characters, or the
    breathtaking cinematography, it delivered an unforgettable experience. What
    kind of movie are you referring to?
  補完 3:
    The movie was full of **unexpected twists**, **emotional depth**, and
    **stunning visuals** that kep

## 92. 予測されたテキストの確率を計算

“The movie was full of"に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。

In [8]:
import textwrap
import tiktoken
import os
import math # logprob（対数確率）を通常の確率に変換するためにmathモジュールを使用

# --- テキスト生成と確率取得関数 ---
def generate_text_with_probabilities(client, model, prompt, max_tokens):
    """
    プロンプトに続くテキストを生成し、各生成トークンの確率を返します。

    Parameters:
        - client: 初期化済みの AzureOpenAI クライアントインスタンス
        - model (str): Azureでデプロイしたモデルのデプロイメント名
        - prompt (str): 入力プロンプト
        - max_tokens (int): 生成する最大トークン数 (1以上)

    Returns:
        tuple: (生成されたテキスト文字列, 各トークンと確率のリスト)。
               エラー発生時は (None, None)。
    """
    if client is None:
        print("エラー: OpenAI client is not initialized. Cannot make API call.")
        return None, None

    try:
        # API呼び出し: テキスト生成とトークン確率の取得
        # temperature は 0.0 (決定論的) を推奨 - 最も尤もらしいシーケンスが生成される
        # n=1 で単一の補完を取得
        # logprobs=True で生成トークンそれぞれの確率を要求
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=0.0, # 確率を観察しやすくするため決定論的にする
            n=1,             # 1つの補完を取得
            logprobs=True    # 生成トークンの確率情報を取得
            # top_p, top_logprobs は今回は使用しない
        )

        # print(f"APIレスポンス: {response}") # 必要に応じてデバッグ用にコメント解除

        response_dict = response.model_dump()

        # 生成されたテキスト全体を取得 (choices[0] は n=1 なので最初の要素)
        generated_text = response_dict["choices"][0]["message"]["content"]

        # logprobs 情報があるか確認
        # logprobs=True にしても、生成テキストがない場合など content が None になる可能性あり
        logprobs_info = response_dict["choices"][0].get("logprobs")
        if not logprobs_info or not logprobs_info.get("content"):
             print("警告: APIレスポンスにlogprobs情報が含まれていませんでした。")
             # テキスト自体は返せたかもしれないが、確率は不明
             return generated_text, []

        # 生成された各トークンとそのlogprobを取得
        token_logprob_list = logprobs_info["content"]

        # 各トークンと確率のリストを作成
        token_probability_list = []
        for token_info in token_logprob_list:
             token_text = token_info["token"]
             logprob = token_info["logprob"]
             # logprob (対数確率) を通常の確率に変換
             # OpenAI API の logprob は自然対数 (底 e) です
             probability = math.exp(logprob)
             # 確率を小数点以下5桁で丸める
             token_probability_list.append((token_text, round(probability, 5)))

        return generated_text, token_probability_list

    except Exception as e:
        print(f"API呼び出し中にエラーが発生しました: {e}")
        # エラー発生時はテキストも確率も取得できていない
        return None, None

# --- 主処理 ---

model_name = "gpt-4o" # あなたのAzure OpenAIでデプロイしたモデルのデプロイメント名を指定
prompt_text = "The movie was full of"
generate_max_tokens_limit = 30 # 生成を打ち切る最大トークン数（適当な長さ）

print(f"=== 課題 ===")
print(f"「'{prompt_text}'」に続くテキストを予測し、生成された各トークンの尤度を表示する。")
print(f"モデル: {model_name}")
print(f"生成最大トークン数: {generate_max_tokens_limit}")
print("============")

# --- 元のプロンプトのトークン化を表示 (参考情報として維持) ---
print("\n--- 元のプロンプトのトークン化結果 ---")
# tiktoken は API クライアント初期化の成功とは独立して動作
# ただし、モデル名がtiktokenでサポートされている必要がある
try:
    encoding = tiktoken.encoding_for_model(model_name)
    input_token_ids = encoding.encode(prompt_text)
    input_tokens_text = [encoding.decode([token_id]) for token_id in input_token_ids]

    print(f"元のプロンプト: '{prompt_text}'")
    print(f"トークンID列: {input_token_ids}")
    print(f"トークンテキスト列: {input_tokens_text}")
    print(f"トークン数: {len(input_token_ids)}")

except Exception as e:
    print(f"プロンプトのトークン化中にエラーが発生しました: {e}")
    print("モデル名が正しいか、tiktoken がインストールされているか確認してください (`pip install tiktoken`)。")


print("\n=== 生成されたテキストとそのトークン確率 ===")

# テキストを生成し、各トークンの確率を取得
# generate_text_with_probabilities 関数内でクライアント初期化の成功をチェック
generated_text, token_probabilities = generate_text_with_probabilities(
    client, # 初期化されたクライアントを渡す
    model_name,
    prompt_text,
    generate_max_tokens_limit
)

# 結果を表示
if generated_text is not None:
    print(f"\n生成されたテキスト (最大{generate_max_tokens_limit}トークン):")
    # 生成テキスト全体を表示
    print(textwrap.fill(generated_text, width=80, initial_indent='    ', subsequent_indent='    '))

    if token_probabilities:
        print("\n各生成トークンとその確率:")
        # 各トークンと確率を表示
        # 見やすくするために、トークンを repr() で囲み、確率を調整して表示
        # repr() を使うと '\n', '\t', ' ' (スペース) など、トークンの厳密な内容が見える
        for token, probability in token_probabilities:
             display_token = repr(token)
             # 表示を揃えるために幅を指定 (必要に応じて調整)
             print(f"  トークン: {display_token:<15}, 確率: {probability}")
    else:
        print("\nトークン確率情報は取得できませんでした。（API呼び出しエラーまたはlogprobs情報なし）")

=== 課題 ===
「'The movie was full of'」に続くテキストを予測し、生成された各トークンの尤度を表示する。
モデル: gpt-4o
生成最大トークン数: 30

--- 元のプロンプトのトークン化結果 ---
元のプロンプト: 'The movie was full of'
トークンID列: [976, 8249, 673, 3149, 328]
トークンテキスト列: ['The', ' movie', ' was', ' full', ' of']
トークン数: 5

=== 生成されたテキストとそのトークン確率 ===

生成されたテキスト (最大30トークン):
    The movie was full of **unexpected twists** and **emotional depth**, keeping
    the audience engaged from start to finish. It blended moments of intense
    drama

各生成トークンとその確率:
  トークン: 'The'          , 確率: 0.29954
  トークン: ' movie'       , 確率: 0.99663
  トークン: ' was'         , 確率: 0.99999
  トークン: ' full'        , 確率: 1.0
  トークン: ' of'          , 確率: 1.0
  トークン: ' **'          , 確率: 0.57547
  トークン: 'unexpected'   , 確率: 0.53364
  トークン: ' twists'      , 確率: 0.99525
  トークン: '**'           , 確率: 0.31889
  トークン: ' and'         , 確率: 0.94333
  トークン: ' **'          , 確率: 0.96054
  トークン: 'em'           , 確率: 0.57162
  トークン: 'otional'      , 確率: 0.99956
  トークン: ' depth'       , 確率: 0.8926
  トークン

## 93. パープレキシティ

適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

+ The movie was full of surprises
+ The movies were full of surprises
+ The movie were full of surprises
+ The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

In [9]:
import textwrap
import tiktoken
import os
import math
import time


# --- パープレキシティ計算関数 ---
# top_k_for_logprobs のデフォルト値を 20 に修正
def calculate_perplexity(client, model, sentence, top_k_for_logprobs=20, delay_seconds=0.05):
    """
    指定された文のパープレキシティを計算します。
    文中の各トークンについて、その前のトークン列をプロンプトとしてAPIを呼び出し、
    当該トークンの確率を取得します。

    Parameters:
        - client: 初期化済みの AzureOpenAI クライアントインスタンス
        - model (str): Azureでデプロイしたモデルのデプロイメント名
        - sentence (str): パープレキシティを測定する文
        - top_k_for_logprobs (int): APIで取得する上位トークン確率の数。
                                   モデルの制限により最大値は通常20です。
                                   対象トークンがこの中に含まれる必要があります。
        - delay_seconds (float): API呼び出し間の待ち時間（秒）。レート制限対策。

    Returns:
        float: 計算されたパープレキシティ。計算できない場合は float('inf')。
    """
    # APIの top_logprobs の制限値を確認し、指定値がそれを超えていないかチェック（念のため）
    actual_top_k = min(top_k_for_logprobs, 20) # API制限を考慮
    if top_k_for_logprobs > 20:
         print(f"  警告: 指定された top_k_for_logprobs ({top_k_for_logprobs}) はモデルの上限 (20) を超えています。実際には {actual_top_k} を使用します。")


    if client is None:
        print("エラー: OpenAI client is not initialized. Cannot calculate perplexity.")
        return float('inf')

    try:
        encoding = tiktoken.encoding_for_model(model)
        # 文全体をトークンIDに変換
        sentence_token_ids = encoding.encode(sentence)
        num_tokens = len(sentence_token_ids)

        # 1トークン以下の文はパープレキシティ計算が定義されない（あるいは1となる）
        if num_tokens <= 1:
            print(f"警告: 文 '{sentence}' のトークン数が1以下です ({num_tokens}トークン)。パープレキシティは定義されません。")
            return 1.0 if num_tokens == 1 else float('inf') # 1トークンの場合は通常1、0トークンは無限大

        total_logprob = 0.0
        num_terms = 0 # logprobを合計できた項の数

        print(f"  '{sentence}' ({num_tokens}トークン) のパープレキシティを計算中...")

        # 文の2番目のトークンから最後までループ (w_i | w_1...w_{i-1}) を取得するため
        # ループ変数 i は、sentence_token_ids リストにおける次のトークンのインデックス
        # プロンプトは sentence_token_ids[:i] となり、次のトークンは sentence_token_ids[i]
        # i=1 から num_tokens-1 まで回す (sentence_token_ids[1] から sentence_token_ids[num_tokens-1] まで)
        for i in range(1, num_tokens):
            target_token_id = sentence_token_ids[i]
            # 対象トークンをテキストにデコード (APIレスポンスのトークンと比較するため)
            target_token_text = encoding.decode([target_token_id])

            # プロンプトとして使用するトークン列 (対象トークンの直前まで)
            prompt_token_ids = sentence_token_ids[:i]
            prompt_text = encoding.decode(prompt_token_ids)

            # プロンプトが空になる場合（文頭のトークンに対する確率）は、
            # APIのlogprobs機能では直接取得が難しいため、2番目のトークンから開始しています。
            # パープレキシティの定義によっては最初のトークンの確率も考慮しますが、
            # 一般的なLM評価では2番目以降で計算することが多いです。
            # ここでは i=1 から開始するため、最初のトークン(i=0)をプロンプトにした次のトークン(i=1)から評価します。
            if not prompt_text: # 最初のトークンに対する評価の場合（i=0の時だが、ここではi=1から開始）
                 # print(f"  スキップ: 最初のトークン '{target_token_text}' の確率計算はスキップされます。")
                 # num_terms += 1 # 項数に含める場合はコメント解除（分母に影響）
                 continue # ここでは i=1 からループしているので、i=0 のケースは発生しない


            # API呼び出し: プロンプトに続く最初の1トークンの確率を取得
            # temperature=0.0 で最も尤もらしい分布を取得しやすくする
            # top_logprobs で対象トークンが分布に含まれる可能性を高める（ただし上限あり）
            try:
                api_response = client.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "user", "content": prompt_text}
                    ],
                    max_tokens=1, # 次の1トークンのみ予測
                    temperature=0.0, # サンプリングのランダム性をなくす（確率分布は変わらない）
                    logprobs=True, # ログ確率を要求
                    top_logprobs=actual_top_k # 実際の制限値を適用
                )

                # API呼び出し間に遅延を入れる（レート制限対策）
                time.sleep(delay_seconds)

            except Exception as e:
                print(f"  エラー: API呼び出し失敗 (プロンプト: '{prompt_text}'): {e}")
                # API呼び出しに失敗した場合は、計算不可として無限大を返す
                return float('inf')


            # レスポンスからログ確率情報を抽出
            logprobs_info = api_response.choices[0].logprobs
            # logprobs_info.content は生成トークンに関するリスト。max_tokens=1なので通常は要素が1つ。
            # logprobs_info.content[0].top_logprobs が上位候補リスト。
            # top_logprobs_list の各要素は TopLogprob オブジェクトであることに注意
            if not logprobs_info or not logprobs_info.content or not logprobs_info.content[0].top_logprobs:
                print(f"  警告: APIレスポンスに十分な logprobs 情報が含まれていませんでした (プロンプト: '{prompt_text}')。")
                 # logprob 情報がなければ計算不可
                return float('inf')


            top_logprobs_list = logprobs_info.content[0].top_logprobs

            # 上位候補リストから、対象トークンの logprob を探す
            found_logprob = None
            # APIレスポンスのトークンテキストと、tiktokenでデコードしたトークンテキストを比較
            # repr() を使うとスペースや改行などの非表示文字を含めて比較できるためより安全
            target_token_repr = repr(target_token_text)

            for token_info in top_logprobs_list:
                 # ここを修正: ドット記法 (.) で属性にアクセスする
                 if repr(token_info.token) == target_token_repr:
                     found_logprob = token_info.logprob # ここを修正
                     break # 見つかったらループを抜ける


            # 対象トークンの logprob が上位K個の中に見つかったか確認
            if found_logprob is not None:
                total_logprob += found_logprob
                num_terms += 1
                # print(f"    Found '{target_token_text}' ({target_token_repr}): {found_logprob}") # デバッグ用
            else:
                # 対象トークンが上位K個に含まれていない場合、その確率は非常に低い
                # ここでは警告を表示し、非常に小さい確率（大きい負のlogprob）を代用する
                fallback_logprob = -20.0 # 代用する大きな負のlogprob
                total_logprob += fallback_logprob
                num_terms += 1
                print(f"  警告: 対象トークン '{target_token_text}' ({target_token_repr}) が上位 {actual_top_k} 候補に見つかりませんでした (プロンプト: '{prompt_text}')。代用logprob ({fallback_logprob}) を使用します。")
                # 見つからなかった時点で計算不可とするなら以下のコメントを解除
                # return float('inf')


        # パープレキシティの計算
        # Perplexity = exp(- (1/N) * Sum(log P))
        # ここで N は合計したlogprobの数 (num_terms)
        if num_terms == 0:
            print("  エラー: logprobを計算できたトークンがありません。")
            return float('inf')

        average_logprob = total_logprob / num_terms
        perplexity = math.exp(-average_logprob)

        return perplexity

    except Exception as e:
        print(f"  致命的なエラーが発生しました: {e}")
        return float('inf')


# --- 主処理 ---

# パープレキシティを測定する文のリスト
sentences_to_test = [
    "The movie was full of surprises",       # 文法的にも意味的にも自然
    "The movies were full of surprises",     # 主語・動詞が複数形で一致、自然
    "The movie were full of surprises",      # 主語が単数、動詞が複数形で不一致 (文法ミス)
    "The movies was full of surprises",      # 主語が複数、動詞が単数形で不一致 (文法ミス)
    "Surprises full of was movie The",       # 単語順序が不自然 (文法ミス)
    "This is a very common sentence",        # 別の自然な文
    "This very is a common sentence",        # 単語順序が不自然 (文法ミス)
    "The quick brown fox jumps over the lazy dog", # より長い標準的な文
]

model_name = "gpt-4o" # あなたのAzure OpenAIでデプロイしたモデル名を指定
# APIの制限に合わせて top_k_for_logprobs を 20 に修正
top_k_for_logprobs = 20
api_delay_seconds = 0.01 # API呼び出し間の短い遅延 (推奨)

print(f"=== 課題 ===")
print(f"与えられた文に対するパープレキシティを測定し、比較する。")
print(f"モデル: {model_name}")
print(f"logprob取得のための上位K (API制限): {top_k_for_logprobs}")
print(f"API遅延: {api_delay_seconds}秒")
print("============")

# クライアントが初期化されているか確認
if client is None:
    print("\nエラー: Azure OpenAI クライアントが初期化されていません。パープレキシティ計算を実行できません。")
else:
    print("\n=== パープレキシティ測定結果 ===")
    # 各文に対してパープレキシティを計算し表示
    for sentence in sentences_to_test:
        # calculate_perplexity 関数内でクライアント初期化の成功をチェック
        perplexity_value = calculate_perplexity(client, model_name, sentence, top_k_for_logprobs, api_delay_seconds)

        if perplexity_value == float('inf'):
            print(f"文: '{sentence}'")
            print(f"パープレキシティ: 計算不可") # エラーメッセージは関数内で出力済み
        else:
            print(f"文: '{sentence}'")
            print(f"パープレキシティ: {perplexity_value:.4f}") # 小数点以下4桁で表示
        print("-" * 30) # 区切り線

=== 課題 ===
与えられた文に対するパープレキシティを測定し、比較する。
モデル: gpt-4o
logprob取得のための上位K (API制限): 20
API遅延: 0.01秒

=== パープレキシティ測定結果 ===
  'The movie was full of surprises' (6トークン) のパープレキシティを計算中...
  警告: 対象トークン ' movie' (' movie') が上位 20 候補に見つかりませんでした (プロンプト: 'The')。代用logprob (-20.0) を使用します。
  警告: 対象トークン ' was' (' was') が上位 20 候補に見つかりませんでした (プロンプト: 'The movie')。代用logprob (-20.0) を使用します。
  警告: 対象トークン ' full' (' full') が上位 20 候補に見つかりませんでした (プロンプト: 'The movie was')。代用logprob (-20.0) を使用します。
  警告: 対象トークン ' of' (' of') が上位 20 候補に見つかりませんでした (プロンプト: 'The movie was full')。代用logprob (-20.0) を使用します。
  警告: 対象トークン ' surprises' (' surprises') が上位 20 候補に見つかりませんでした (プロンプト: 'The movie was full of')。代用logprob (-20.0) を使用します。
文: 'The movie was full of surprises'
パープレキシティ: 485165195.4098
------------------------------
  'The movies were full of surprises' (6トークン) のパープレキシティを計算中...
  警告: 対象トークン ' movies' (' movies') が上位 20 候補に見つかりませんでした (プロンプト: 'The')。代用logprob (-20.0) を使用します。
  警告: 対象トークン ' were' (' were') が上位 20 候補に見つかりませんでし

## 94. チャットテンプレート

"What do you call a sweet eaten after dinner?"という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

## 95. マルチターンのチャット

問題94で生成された応答に対して、追加で"Please give me the plural form of the word with its spelling in reverse order."と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

## 96. プロンプトによる感情分析

事前学習済み言語モデルで感情分析を行いたい。テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)の開発データにおける正解率を測定せよ。

## 97. 埋め込みに基づく感情分析

事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。

## 98. ファインチューニング

問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。

## 99. 選好チューニング

問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。
